<a href="https://colab.research.google.com/github/famemov/ml-student-stress-prediction/blob/main/students_stress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

# ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ

In [26]:
# Загрузка датасета
try:
    df = pd.read_csv('student_stress_sleep_screen.csv')
    print("Датасет успешно загружен.")
except FileNotFoundError:
    print("Файл не найден! Пожалуйста, загрузите файл 'student_stress_sleep_screen.csv' в сессию Colab.")
    # Раскомментируйте строчки ниже, если нужно загрузить файл вручную
    # from google.colab import files
    # uploaded = files.upload()
    # df = pd.read_csv(list(uploaded.keys())[0])

# Выбираем признаки (согласно теме: учебная нагрузка и сон/экран)
features = ['study_hours', 'sleep_hours', 'screen_time_hours']
target = 'stress_level'

# Проверка на пропуски
df = df.dropna(subset=features + [target])

X = df[features]
y = df[target]

# Кодирование целевой переменной (Low -> 0, Medium -> 1, High -> 2)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
class_names = le.classes_
print(f"Классы целевой переменной: {class_names}")

# Стандартизация признаков (важно для нейросети и KNN)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

Датасет успешно загружен.
Классы целевой переменной: ['High' 'Low' 'Medium']


# ОБУЧЕНИЕ 4-х КЛАССИЧЕСКИХ АЛГОРИТМОВ

In [27]:
models_results = {}

# 1. Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
acc_dt = accuracy_score(y_test, dt.predict(X_test))
models_results['Decision Tree'] = acc_dt

# 2. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
acc_rf = accuracy_score(y_test, rf.predict(X_test))
models_results['Random Forest'] = acc_rf

# 3. SVM (Support Vector Machine)
svm = SVC(probability=True, random_state=42)
svm.fit(X_train, y_train)
acc_svm = accuracy_score(y_test, svm.predict(X_test))
models_results['SVM'] = acc_svm

# 4. KNN (K-Nearest Neighbors)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
acc_knn = accuracy_score(y_test, knn.predict(X_test))
models_results['KNN'] = acc_knn

# НЕЙРОННАЯ СЕТЬ (TENSORFLOW) - 5-й алгоритм

In [28]:
def create_model(hidden_layers=2, neurons=64, dropout_rate=0.3, learning_rate=0.001):
    model = Sequential()
    # Входной слой
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

    # Скрытые слои
    for _ in range(hidden_layers - 1):
        model.add(Dense(neurons, activation='relu'))
        model.add(Dropout(dropout_rate))

    # Выходной слой (3 нейрона для 3 классов: Low, Medium, High)
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy', # Для классов 0, 1, 2
                  metrics=['accuracy'])
    return model

# Гиперпараметры
hidden_layers = 3
neurons = 64
dropout_rate = 0.2
learning_rate = 0.001

model = create_model(hidden_layers, neurons, dropout_rate, learning_rate)

# Обучение
print("\nЗапуск обучения нейросети...")
history = model.fit(X_train, y_train,
                    epochs=4,
                    batch_size=16,
                    validation_data=(X_test, y_test),
                    verbose=0)

# Оценка нейросети
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
models_results['Neural Network'] = test_acc
print(f"Neural Network Test Accuracy: {test_acc:.2f}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




Запуск обучения нейросети...
Neural Network Test Accuracy: 0.95


# ВИЗУАЛИЗАЦИЯ

In [29]:
# График 1: История обучения (Loss и Accuracy)
fig_hist = px.line(history.history,
                   y=['loss', 'val_loss', 'accuracy', 'val_accuracy'],
                   title='Динамика обучения нейросети (Loss и Accuracy)')
fig_hist.show()

# График 2: Сравнение всех 5 моделей
results_df = pd.DataFrame(list(models_results.items()), columns=['Model', 'Accuracy'])
fig_bar = px.bar(results_df, x='Model', y='Accuracy',
                 title='Сравнение точности 5 алгоритмов',
                 color='Accuracy', text_auto='.2f')
fig_bar.show()

# График 3: Матрица ошибок (Confusion Matrix) для Нейросети
y_pred_nn_probs = model.predict(X_test)
y_pred_nn = np.argmax(y_pred_nn_probs, axis=1)

cm = confusion_matrix(y_test, y_pred_nn)
fig_cm = px.imshow(cm, text_auto=True,
                   labels=dict(x="Предсказано", y="Истинно", color="Кол-во"),
                   x=class_names, y=class_names,
                   title="Матрица ошибок (Neural Network)")
fig_cm.show()

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step


# ИНТЕРАКТИВНОЕ ПРИЛОЖЕНИЕ

In [30]:
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np

# Настройка стилей виджетов
style = {'description_width': 'initial'}
layout = widgets.Layout(width='50%')

# Создаем слайдеры
study_widget = widgets.FloatSlider(value=5, min=0, max=15, step=0.5, description='Часы учебы (Study Hours):', style=style, layout=layout)
sleep_widget = widgets.FloatSlider(value=7, min=3, max=12, step=0.5, description='Часы сна (Sleep Hours):', style=style, layout=layout)
screen_widget = widgets.FloatSlider(value=4, min=0, max=12, step=0.5, description='Экран (Screen Time):', style=style, layout=layout)

output = widgets.Output()

def predict_stress(b):
    with output:
        output.clear_output()

        # 1. Подготовка данных
        # Важно: порядок переменных должен совпадать с тем, как учили (study, sleep, screen)
        # Если в features порядок другой, поменяйте здесь переменные местами
        input_data = np.array([[study_widget.value, sleep_widget.value, screen_widget.value]])

        # Масштабируем данные тем же склером, что и при обучении
        input_scaled = scaler.transform(input_data)

        # 2. Получение предсказания
        pred_probs = model.predict(input_scaled, verbose=0)[0] # Получаем массив вероятностей [prob_0, prob_1, prob_2]
        pred_class_idx = np.argmax(pred_probs)
        pred_class = class_names[pred_class_idx]

        # 3. Формирование красивого HTML вывода с полосками

        # Определяем цвета для классов для наглядности
        colors = {"Low": "#4CAF50", "Medium": "#FF9800", "High": "#F44336"} # Зеленый, Оранжевый, Красный

        bars_html = ""
        for name, prob in zip(class_names, pred_probs):
            percent = prob * 100
            color = colors.get(name, "#2196F3")

            # HTML для полоски прогресса
            bars_html += f"""
            <div style="margin-bottom: 8px;">
                <div style="display: flex; justify-content: space-between;">
                    <b>{name}</b>
                    <span>{percent:.2f}%</span>
                </div>
                <div style="width: 100%; background-color: #e0e0e0; border-radius: 5px; height: 10px;">
                    <div style="width: {percent}%; background-color: {color}; height: 10px; border-radius: 5px;"></div>
                </div>
            </div>
            """

        final_html = f"""
        <div style="border: 2px solid #333; padding: 15px; border-radius: 10px; max-width: 600px;">
            <h3 style="margin-top: 0;">Результат прогноза:</h3>
            <p style="font-size: 18px;">Наиболее вероятный уровень стресса: <b style="color: {colors.get(pred_class, 'black')}">{pred_class}</b></p>
            <hr>
            <h4>Детализация вероятностей:</h4>
            {bars_html}
        </div>
        """

        display(HTML(final_html))

# Кнопка
button = widgets.Button(description="Рассчитать вероятность", button_style='primary', layout=widgets.Layout(width='30%'))
button.on_click(predict_stress)

# Вывод интерфейса
print("\n--- ИНТЕРАКТИВНЫЙ КАЛЬКУЛЯТОР СТРЕССА ---")
display(widgets.VBox([study_widget, sleep_widget, screen_widget, button, output]))


--- ИНТЕРАКТИВНЫЙ КАЛЬКУЛЯТОР СТРЕССА ---


# HTML ОТЧЕТ

In [31]:
html_summary = f"""
<div style="border:2px solid #4CAF50; padding: 10px; border-radius: 5px;">
    <h2>Итоговый отчет</h2>
    <p><b>Лучшая модель:</b> {max(models_results, key=models_results.get)} ({max(models_results.values()):.2f})</p>
    <p><b>Конфигурация НС:</b> Слоев: {hidden_layers}, Нейронов: {neurons}, Dropout: {dropout_rate}</p>
    <p><b>Использованные признаки:</b> {', '.join(features)}</p>
</div>
"""
display(HTML(html_summary))